In [ ]:
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')

import os
import random
from tqdm import tqdm

import timm
import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import pandas as pd
import cv2

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, GroupKFold

from torch.utils.data import Dataset

In [ ]:
config = {
    "fold_num": 5,
    "seed": 6,
    "model_arch": "tf_efficientnet_b3",
    "model_shape" : "eff",
    "debug": 1,
    "pretrained" : 1,
    "img_size": 256,
    "epochs": 1,
    "train_bs": 16,
    "valid_bs": 16,
    "T_0": 10,
    "lr": 3e-4,
    "min_lr": 3e-6,
    "weight_decay": 1e-4,
    "num_workers": 4,
    "verbose_step": 1,
    "device": "cuda:0",
    "tta": 2,
    "monitor" : "val_accuracy",
    "patience" : 2,
    "mode" : "max"
}

In [ ]:
def seed_everything(seed):
    "seed値を一括指定"
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_img(path):
    """
    pathからimageの配列を得る
    """
    im_bgr = cv2.imread(path)
    if im_bgr is None:
        print(path)
    im_rgb = im_bgr[:, :, ::-1]
    
    return im_rgb

In [ ]:
seed_everything(config['seed'])
device = torch.device(config['device'])

In [ ]:
# data読み込み
df_train = pd.read_csv("/kaggle/input/comp-dog/train.csv")
df_test = pd.read_csv("/kaggle/input/comp-dog/submission.csv")
submission = pd.read_csv("/kaggle/input/comp-dog/submission.csv")

In [ ]:
# ラベルエンコーディング、130個
le = preprocessing.LabelEncoder()
df_train["label"] = le.fit_transform(df_train["class"])
df_train["path"] = "/kaggle/input/comp-dog/train/train/" + df_train["img_name"] + ".jpg"
df_test["label"] = -1
df_test["path"] = "/kaggle/input/comp-dog/test/test/" + df_test["img_name"] + ".jpg"
print("training_class : ", len(le.classes_))

In [ ]:
# data augmentation
from albumentations import (
    PadIfNeeded, HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize,ToGray
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms(input_shape):
    return Compose([
            Resize(input_shape, input_shape),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_valid_transforms(input_shape):
    return Compose([
                Resize(input_shape, input_shape),
                Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
                ToTensorV2(p=1.0),
            ], p=1.)

In [ ]:
# Dataset
class DogDataset(Dataset):
    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):
        target = self.df.loc[index]["label"]

        img  = get_img(self.df.loc[index]["path"])

        if self.transforms:
            img = self.transforms(image=img)['image']

        return img, target

In [ ]:
# Model
class DogClassifier(nn.Module):
    def __init__(self, model_arch, n_class, model_shape, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)

        if model_shape == "eff":
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, n_class)
        elif model_shape == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, n_class)
        elif model_shape == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x

def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, verbose_step, scheduler=None, schd_batch_update=False, arcface=False):
    model.train()
    scaler = GradScaler()

    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        with autocast():
            image_preds = model(imgs)
            loss = loss_fn(image_preds, image_labels)

            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None and schd_batch_update:
                scheduler.step()

            if ((step + 1) % verbose_step == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'
                pbar.set_description(description)

    print("train: "+ description)
    if scheduler is not None and not schd_batch_update:
        scheduler.step()

def valid_one_epoch(epoch, model, loss_fn, val_loader, device, verbose_step, scheduler=None, schd_loss_update=False):
    model.eval()

    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []

    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        image_preds = model(imgs)

        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]

        loss = loss_fn(image_preds, image_labels)

        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]

        if ((step + 1) % verbose_step== 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)

    print("valid "+ description)
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))

    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()

    monitor = {}
    monitor["val_loss"] = loss_sum/sample_num
    monitor["val_accuracy"] = (image_preds_all==image_targets_all).mean()
    return monitor

def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs, _) in pbar:
        imgs = imgs.to(device).float()

        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]

    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

class EarlyStopping:
    def __init__(self, patience):
        self.max_val_monitor = 1000
        self.min_val_monitor = -1000
        self.val_epoch = -1
        self.stop_count = 0
        self.patience = patience
        self.min_delta = 0

    # mode = "min" or "max"(val_loss, val_accuracy)
    def update(self, monitor, epoch, mode):
        if mode == "max":
            if monitor > self.min_val_monitor:
                self.min_val_monitor = monitor
                self.val_epoch = epoch
                self.stop_count = 0
            else:
                self.stop_count+=1
        else:
            if monitor < self.max_val_monitor:
                self.max_val_monitor = monitor
                self.val_epoch = epoch
                self.stop_count = 0
            else:
                self.stop_count+=1

        if self.stop_count >= self.patience:
            return -1
        else:
            return 0

In [ ]:
test_ds = DogDataset(df_test, transforms=get_valid_transforms(config["img_size"]))
test_loader = torch.utils.data.DataLoader(
    test_ds,
    batch_size=config["valid_bs"],
    num_workers=config["num_workers"],
    shuffle=False,
    pin_memory=True,
)

In [ ]:
val_epochs = []
folds = StratifiedKFold(
            n_splits=config['fold_num'],
            shuffle=True,
            random_state=config['seed']).split(np.arange(df_train.shape[0]),
            df_train.label.values
        )

for fold, (trn_idx, val_idx) in enumerate(folds):
    if fold > 0: # 時間がかかるので最初のモデルのみ
        break

    # Dataset作成
    train_ = df_train.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df_train.loc[val_idx,:].reset_index(drop=True)

    train_ds = DogDataset(train_, transforms=get_train_transforms(config["img_size"]))
    valid_ds = DogDataset(valid_, transforms=get_valid_transforms(config["img_size"]))

    train_loader = torch.utils.data.DataLoader(
            train_ds,
            batch_size=config["train_bs"],
            pin_memory=True,
            drop_last=False,
            num_workers=config["num_workers"],
        )

    valid_loader = torch.utils.data.DataLoader(
        valid_ds,
        batch_size=config["valid_bs"],
        num_workers=config["num_workers"],
        shuffle=False,
        pin_memory=True,
    )
    
    #modelと最適化関数作成
    model = DogClassifier(config['model_arch'],
                              len(le.classes_),
                              config["model_shape"],
                              pretrained=config["pretrained"])

    model.eval()
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=config['T_0'], T_mult=1, eta_min=config['min_lr'], last_epoch=-1)
    er = EarlyStopping(config['patience'])

    loss_tr = nn.CrossEntropyLoss().to(device)
    loss_vl = nn.CrossEntropyLoss().to(device)
    
    print(f'Training with fold {fold} started (train:{len(trn_idx)}, val:{len(val_idx)})')
    # train phase
    for epoch in range(config['epochs']):
        train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, config['verbose_step'],scheduler=scheduler, schd_batch_update=False)
        with torch.no_grad():
            monitor = valid_one_epoch(epoch, model, loss_vl, valid_loader, device, config['verbose_step'], scheduler=None, schd_loss_update=False)

        # Early Stopiing
        if er.update(monitor[config["monitor"]], epoch, config["mode"]) < 0:
            break

        if epoch == er.val_epoch:
            torch.save(model.state_dict(),f'/kaggle/working/{config["model_arch"]}_fold_{fold}_{epoch}')

    val_epochs.append(er.val_epoch)
    
    del model, train_loader, valid_loader, optimizer, scheduler
    
    #inference phase
    model = DogClassifier(config['model_arch'],
                              len(le.classes_),
                              config["model_shape"]).to(device)

    model.load_state_dict(torch.load(f'/kaggle/working/{config["model_arch"]}_fold_{fold}_{er.val_epoch}'))
    tst_preds = []
    with torch.no_grad():
        tst_preds += [inference_one_epoch(model, test_loader, device)]
    tst_preds = np.mean(tst_preds, axis=0)
    tst_pred = le.inverse_transform(np.argmax(tst_preds, axis=1))
    submission["class"] = tst_pred
    submission.to_csv(f"/kaggle/working/benchmark.csv", index=False)